In [ ]:
!pip install python-docx

In [ ]:
import pandas as pd
import numpy as np
import warnings
import random
from multiprocessing import Process, Manager
warnings.filterwarnings("ignore")

available_players = ["nsilva", "guest_cgomes", "guest_gregs", 
                     "rpinheiro", "ggomes", "guest_jpab", 
                     "pduarte", "jsilva", 'apimenta', 'guest_tymoschuk']

anygoal_data = pd.read_csv("anygoal_season_2022_resume.csv")

anybrain_df = pd.read_excel("players_03_23.xlsx", engine='openpyxl')

player_stats = ['Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade',
                'Fisico', 'GK', 'Overall', 'Overall_No_GK', 'Point_System']

In [ ]:
def add_guest_player(anybrain_df):
    # Get the new players
    new_players = list(set(available_players) - set(anybrain_df.Nome.tolist()))
    # Get the similar player
    similar_players = [input("Who does {} look like? ".format(player)) for player in new_players]
                         
    for player, guest in zip(similar_players, new_players):
        # Create a copy of the similar player
        player_data = anybrain_df[anybrain_df.Nome == player].copy()
        # Change the player name
        player_data.loc[(player_data.Nome == player), "Nome"] = guest
        # Add the new row
        anybrain_df = anybrain_df.append(player_data.to_dict('records')[0], ignore_index=True)
        
        # Give any note on the player
        player_note = input("Any note on the player? ").upper().split()
        if (any([True if (x not in player_note) else False for x in ["NO", "NONE", "NEGATIVE"]])):
            for stat in player_stats[:-3]:
                # Check what situation occurs
                if ((stat.upper() in player_note) and ('Better'.upper() in player_note)):
                    anybrain_df.loc[(anybrain_df.Nome == guest), stat] += 1

                elif ((stat.upper() in player_note) and ('Worse'.upper() in player_note)):
                    anybrain_df.loc[(anybrain_df.Nome == guest), stat] -= 1
        
                else:
                    pass
                
        # Upda the overall values
        aux = anybrain_df[(anybrain_df.Nome == guest)]
        anybrain_df.loc[(anybrain_df.Nome == guest), 'Overall'] = aux[player_stats[:-3]].mean(axis=1)
        anybrain_df.loc[(anybrain_df.Nome == guest), 'Overall_No_GK'] = aux[player_stats[:-4]].mean(axis=1)
    
    return anybrain_df, new_players
        
def treat_and_merge(anybrain_df, anygoal_data):
    anybrain_df.set_index('Unnamed: 0', inplace=True)
    anybrain_df = anybrain_df.T.reset_index()
    anybrain_df.columns = ['Nome', 'Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade', 
                           'Fisico', 'GK', 'Overall', 'Overall_No_GK']
    anybrain_df = anybrain_df[~anybrain_df.Nome.str.contains("Unnamed:")]
    anybrain_df.index.name = None
    anybrain_df.Nome.replace({'Jpab' : 'guest_jpab', 'J lopes' : 'guest_jlopes', 'carlos' : 'guest_cgomes', 
                              'Mineiro' : 'nsilva', 'Nelo' : 'esousa', 'Ze' : 'jsilva', 'Fintas' : 'spinto', 
                              'Pimenta' : 'apimenta', 'Xerife' : 'rpinheiro', 'Champion' : 'pduarte', 
                              'Gus' : 'ggomes', 'Costeira' : 'gcosteira', 'Flash' : 'guest_flash', 
                              'random' : 'guest_random', "anguyen" : 'ahn-vu', "gregs" : "guest_gregs"}, inplace=True)
    
    # Add the guest player to the bd, and his values will be the most similar to a specific player
    anybrain_df, new_players = add_guest_player(anybrain_df)
    for player in new_players:
        if (player not in anygoal_data.Player.unique()):
            anygoal_data = anygoal_data.append({'Player' : "{}".format(player), 'Games' : 0, 'MVP' : 0, 'MVP_%' : 0,
                                                'Victories' : 0, 'Losses' : 0, 'Victories_%' : 0, 'Goals' : 0,
                                                'Goals/Games' : 0, 'Team_Goals' : 0, 'Team_Suffered' : 0,
                                                'Team_Influence' : 0, 'Team_Goals/Games' : 0, 'Team_Suffered/Games' : 0,
                                                'Biggest_W_Streak' : 0, 'Biggest_L_Streak' : 0, 'Point_System' : 0}, ignore_index = True)

    # Merge the anybrain_df with anygoal
    merged_df = anybrain_df.merge(anygoal_data, left_on='Nome', right_on='Player')
    merged_df.drop(['Unnamed: 0', 'Player', 'MVP_%', 'Victories', 'Losses', 'Victories_%', 
                    'Goals/Games', 'Team_Goals', 'Team_Suffered', 'Team_Influence', 'Team_Goals/Games', 
                    'Team_Suffered/Games', 'Biggest_W_Streak', 'Biggest_L_Streak'], axis=1, inplace=True)
    
    return merged_df

In [ ]:
merged_df = treat_and_merge(anybrain_df, anygoal_data)

In [ ]:
data_players = merged_df.loc[merged_df.Nome.isin(available_players), :]
data_players

In [ ]:
def split_df(df):
    half_length = df.shape[0] // 2
    df1 = df.iloc[:half_length, :]
    df2 = df.iloc[half_length:, :]
    return df1, df2

def teams_conditions(mean_1, mean_2):   
    mean_1_2 = mean_1.subtract(mean_2).abs()
    mean_condition = [(mean_1_2.Point_System < 0.4)]+[(mean_1_2[x]< 0.35) for x in player_stats[:-1]]
    return mean_condition, mean_1_2[player_stats].sum()

def balance_dfs(df1, df2, counter, final_list):
    for _ in range(counter):
        # Generate a random value
        i1 = np.random.randint(df1.shape[0])
        i2 = np.random.randint(df2.shape[0])
        
        # Switch players based on the random index
        row1, row2 = df1.iloc[i1,:], df2.iloc[i2,:]
        df1.iloc[i1, :] = row2.values
        df2.iloc[i2, :] = row1.values
        
        # Calculate the mean & the team conditions
        df_mean1 = df1.mean()
        df_mean2 = df2.mean()
        condition, sum_difference = teams_conditions(df_mean1, df_mean2)
        val = sum(bool(x) for x in condition)

        A = df1.Nome.values.tolist()
        A.sort()
        
        B = df2.Nome.values.tolist()
        B.sort()
        
        # Append the iteration dict to the shared list
        final_list.append({"Sum_Abs_Difference" : sum_difference,
                           "N_Conditions" : val,
                           "A_team" : ' '.join(A),
                           "B_team" : ' '.join(B)})
        

def parallel_algorithm(df, n_process=8):
    # Split the dataframe into 2
    df1, df2 = split_df(df)
    
    # Split the iterations between the available processes
    counter = int(4000/n_process)
    
    # Create a list that will contain the final dataframe
    final_list = Manager().list([])
    processes = []

    for _ in range(n_process):
        # Set the process to work on the "balance_dfs" function
        p = Process(target=balance_dfs, args=(df1, df2, counter, final_list))
        processes.append(p)
        p.start()

    # completing process
    for p in processes:
        p.join()
        
    return pd.DataFrame.from_records(final_list).drop_duplicates(subset=['A_team'])

In [ ]:
df = parallel_algorithm(data_players, n_process=8)
df.to_csv('combination.csv', index=False)

In [ ]:
def get_best_combination(df):
    max_conditions = df[df["N_Conditions"] == df["N_Conditions"].max()]
    min_difference = max_conditions[max_conditions["Sum_Abs_Difference"] == max_conditions["Sum_Abs_Difference"].min()]
    
    return min_difference["A_team"].iloc[0].split(), min_difference["B_team"].iloc[0].split()

In [ ]:
get_best_combination(df)

In [ ]:
A, B = get_best_combination(df)

tmp = ["Jogador_1", "Jogador_2", "Jogador_3", "Jogador_4", "Jogador_5", "Jogador_6", "Jogador_7", "Jogador_8", "Jogador_9", "Jogador_10"]
tmp2 = A + B

full_players = dict(zip(tmp,tmp2))


In [ ]:
import os 

# Meti isto a apagar o ficheiro caso já existe, porque se existir, isto vai dar um erro qualquer de permissões 

if os.path.exists("./Anygoal_Template.docx"):
    os.remove("./Anygoal_Template.docx")

In [ ]:
import docx
from docx.shared import Pt


doc = docx.Document("./template/Anygoal_Template.docx")


for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            if cell.text in full_players:
                cell.text = cell.text.replace(cell.text, full_players[cell.text])


doc.save("./Anygoal_Template.docx")
